In [26]:
#!pip install BERTopic
#import sklearn.datasets 
#from sklearn.datasets import fetch_20newsgroups
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
#!pip install scipy


In [27]:
from bertopic import BERTopic
import pandas as pd
import pickle
from typing import List
from transformers import pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objs as go


In [ ]:
df=pd.read_csv("/content/AI.csv")
df = df.dropna()
df["new"] = df["Title"].astype(str) + "" + df["Abstract"].astype(str)
docs = df["new"].tolist() # Convert the column to a list of strings



In [ ]:
#unigram
topic_model = BERTopic(calculate_probabilities=True,verbose=True,)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/177 [00:00<?, ?it/s]

2023-03-03 01:14:36,601 - BERTopic - Transformed documents to Embeddings
2023-03-03 01:15:04,285 - BERTopic - Reduced dimensionality
2023-03-03 01:15:05,852 - BERTopic - Clustered reduced embeddings


In [ ]:
# Save fitted model and data in a pickle file
with open('topic_model.pkl', 'wb') as f:
    pickle.dump((topic_model, docs), f)

In [3]:
# Load saved model and data from pickle file
with open('topic_model.pkl', 'rb') as f:
    topic_model, docs = pickle.load(f)

In [4]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
from random import sample

tokenizer = T5Tokenizer.from_pretrained("t5-base")
language_model = T5ForConditionalGeneration.from_pretrained("t5-base")
headline_generator = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")

summarizer = pipeline("summarization", model=language_model, tokenizer=tokenizer, framework="tf")


In [5]:
def generate_topic_label(articles: List[str]) -> str:
    summary_list = []
    current_token_length = 0
    max_token_length = 512
    
    for article in sample(articles, k=len(articles)):
        summary = summarizer(article)[0]["summary_text"]
        current_token_length += len(tokenizer.encode(summary))
        if current_token_length >= max_token_length:
            break
        summary_list.append(summary)

    encoding = tokenizer.encode("headline: " + " ".join(summary_list), return_tensors="pt")
    output = headline_generator.generate(encoding)
    
    return tokenizer.decode(output[0][1:-1])


In [6]:
df=topic_model.get_document_info(docs)
df

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,Smart Cropping based on Predicted Solar Radiat...,-1,-1_the_energy_of_to,the - energy - of - to - and - power - in - is...,0.017993,False
1,Systematic literature review of photovoltaic o...,-1,-1_the_energy_of_to,the - energy - of - to - and - power - in - is...,0.421330,False
2,Demystifying the use of era5-land and machine ...,-1,-1_the_energy_of_to,the - energy - of - to - and - power - in - is...,0.124015,False
3,A CNN-Based Warning Information Preprocessing ...,-1,-1_the_energy_of_to,the - energy - of - to - and - power - in - is...,0.978158,False
4,Short-Term Photovoltaic Power Prediction Model...,-1,-1_the_energy_of_to,the - energy - of - to - and - power - in - is...,0.555617,False
...,...,...,...,...,...,...
5656,Influence of initial pH on thermophilic anaero...,0,0_was_production_were_of,was - production - were - of - hydrogen - biog...,0.437387,False
5657,Carrying capacity assessment of forest resourc...,23,23_biomass_decision_forest_support,biomass - decision - forest - support - and - ...,1.000000,False
5658,The diffusion of a renewable energy technology...,69,69_tis_innovation_diffusion_technological,tis - innovation - diffusion - technological -...,1.000000,False
5659,Technology innovation systems and technology d...,69,69_tis_innovation_diffusion_technological,tis - innovation - diffusion - technological -...,1.000000,False


In [7]:
# define the topic ID to filter
topic_id = 3

# initialize an empty list to store the documents
topic_3 = []

# loop through the rows of the dataframe
for index, row in df.iterrows():
    # check if the 'Topic' value matches the desired topic ID
    if row['Topic'] == topic_id:
        # if yes, append the corresponding 'Documents' value to the list
        topic_3.append(row['Document'])

# print the list of documents corresponding to the topic ID
#print(topic_3)

In [10]:
def run_generate_topic_label(articles: List[str]) -> List[str]:
    labels = []
    for i in range(15):
        label = generate_topic_label(articles)
        labels.append(label)
    return labels

In [9]:
labels = run_generate_topic_label(topic_3)
print(labels)

Your max_length is set to 200, but you input_length is only 105. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
Your max_length is set to 200, but you input_length is only 147. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=73)
Your max_length is set to 200, but you input_length is only 168. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=84)
Your max_length is set to 200, but you input_length is only 178. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=89)
Your max_length is set to 200, but you input_length is only 130. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 200, but you input_len

['Wind Power Forecasting - A New Approach', 'Wind Speed Forecasting - A Neural Network Based Approach', 'Wind Speed Prediction - A Review']


In [14]:
def measure_similarity_of_topic(topic_labels: List[str]):
  embedding = topic_model.embedding_model.embed(topic_labels)
  similarity_matrix = cosine_similarity(embedding)
  
  top_label = topic_labels[np.argmax(np.sum(similarity_matrix, axis=1))]
    
  # scoring topic
  triu_mat = np.triu(similarity_matrix, k=1)
  score = np.mean(triu_mat[np.nonzero(triu_mat)])
  
  return similarity_matrix, top_label, score

In [15]:
similarity=measure_similarity_of_topic(labels)
similarity

(array([[1.0000005 , 0.76726764, 0.73155355],
        [0.76726764, 0.9999999 , 0.7848147 ],
        [0.73155355, 0.7848147 , 0.9999999 ]], dtype=float32),
 'Wind Speed Forecasting - A Neural Network Based Approach',
 0.76121205)

In [23]:
def plot_similarity_matrix(similarity_matrix: np.ndarray, labels: List[str]):
    # Create trace for heatmap
    trace = go.Heatmap(z=similarity_matrix,
                       x=labels,
                       y=labels,
                       colorscale='Viridis')

    # Create layout with hovertext and title
    layout = go.Layout(title='Topic Similarity Matrix',
                       hovermode='closest',
                       hoverlabel=dict(bgcolor='white',
                                       font=dict(size=16)),
                       xaxis=dict(tickangle=-45, tickfont=dict(size=14)),
                       yaxis=dict(tickfont=dict(size=14)))

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)

    # Show figure
    fig.show()


In [24]:
similarity, top_label, score = measure_similarity_of_topic(labels)
plot_similarity_matrix(similarity, labels)


In [25]:
topic_model.get_topic(3)

[('wind', 0.047472681436073),
 ('speed', 0.031077225222894682),
 ('forecasting', 0.015355916357132463),
 ('power', 0.014143417622886196),
 ('neural', 0.012375871458123307),
 ('prediction', 0.01192333723737786),
 ('data', 0.011268927099121836),
 ('the', 0.011177212930248471),
 ('of', 0.011027724786138797),
 ('in', 0.010731572072568153)]